In [ ]:

NUM_PROC_MAP = 150 # expect 30 min with single process
PUSH_TO_HF = False

In [2]:
import os

from dotenv import load_dotenv
load_dotenv()
import datasets
from datasets import DatasetDict
from datasets import load_dataset
import datasets
import torch
import json
from huggingface_hub import HfApi

As in the official [preprocessing repo](https://github.com/babylm/babylm_data_preprocessing), we get the number of words via `line.split()`:

In [3]:

# from nltk.tokenize import RegexpTokenizer
# tokenizer = RegexpTokenizer(r'\w+')
# word_count = lambda d: len([w for w in tokenizer.tokenize(d) if w.isalnum()])
# sum([word_count(d) for d in dataset["text"]])
word_count = lambda d: len(d.split())
word_count_dataset = lambda dataset:sum([word_count(d) for d in dataset["text"]])

## New Curriculum based on Influence during learning on randomly shuffled data

In [4]:
import numpy as np

from scipy.stats import norm
from scipy.stats import f
from scipy.stats import lognorm
import plotting
import matplotlib.pyplot as plt
import json
import os
import pandas as pd
import torch
import datasets
from datasets import load_dataset
import util

HF_USERNAME = None
from huggingface_hub import HfApi
api = HfApi()
if HF_USERNAME is None:
    HF_USERNAME = api.whoami()["name"]
    

In [5]:
def gaussian_filter(size, **args):
    indices = np.arange(-size, size+1, 1)
    weights =  norm.pdf(indices, **args)
    return weights# / np.sum(weights)


def f_filter(size, **args):
    indices = np.arange(-size, size+1, 1)
    print(indices)
    weights = f.pdf(indices+1, **args)
    return weights# / np.sum(weights)
def lognorm_filter(size, **args):
    #print(args)
    indices = np.arange(-size, size+1, 1)
    #print(indices)
    weights = lognorm.pdf(indices+1, **args)
    return weights / np.sum(weights)




dirac_filter = lambda size: np.eye(1, (size*2)+1, size, dtype=float).flatten()





In [6]:

def reweight_df(df, influence_cols, filter_weights):
    scores = pd.DataFrame(np.apply_along_axis(lambda m: np.convolve(m,filter_weights, mode="valid")[1:-1], axis=1, arr=df[influence_cols].to_numpy()))
    scores.columns = influence_cols
    df_reweighted = df.copy()
    df_reweighted[influence_cols] = scores
    df_reweighted["total"] = df_reweighted[influence_cols].sum(axis=1)
    return df_reweighted


def upload(tensor, name):
    p = name + "_" + os.path.basename(model_name) + ".pt"
    if PUSH_TO_HF:
        torch.save(tensor, p)
        api.upload_file(
            path_or_fileobj=p,
            path_in_repo=p,
            repo_id=dataset_name,
            repo_type="dataset"
        )
        os.remove(p)
    else:
        print("skipping upload for", p)


In [7]:
from huggingface_hub import list_models


username = "loris3"

tasks = [(model.id, model.id.replace("_random","")) for model in list_models(author=username) if "random" in model.id]

In [ ]:


# load influence data
for model_name,dataset_name in tasks:

    assert dataset_name in model_name


    RANDOM_CURRICULUM_NAME ="random.pt"


    influence_output_dir = os.path.join("./mean_influence", os.path.basename(model_name), "_".join([os.path.basename(dataset_name) +"_"+"train"]*2))

    dataset = load_dataset(dataset_name)["train"]
    curriculum = util.get_curriculum(dataset_name, RANDOM_CURRICULUM_NAME)


    df = pd.DataFrame({int(result_checkpoint.replace("checkpoint-","")): torch.load(os.path.join(influence_output_dir,result_checkpoint),weights_only=True,map_location="cpu").numpy().flatten() for result_checkpoint in os.listdir(influence_output_dir)})
    df.sort_index(axis=1)

    df = df.reindex(sorted(df.columns, reverse=False), axis=1)
    influence_cols = df.columns
    df["total"] = df.sum(axis=1)
    df[["text", "source","stage"]] = dataset.to_pandas()
    df["document_lenght"] = df["text"].str.split().str.len()




    get_order_full = lambda df: torch.stack([torch.tensor(df[checkpoint].sort_values(ascending=False).index.to_numpy()) for checkpoint in influence_cols])
    get_order_top_n_pct = lambda top_n_pct, df: torch.stack([torch.tensor(df[checkpoint].sort_values(ascending=False).head(int(len(df)*top_n_pct*0.01)).index.to_numpy()) for checkpoint in influence_cols])





    #############

    lognorm_order = get_order_full(reweight_df(df, influence_cols, lognorm_filter(len(influence_cols), s=1, loc=0, scale=0.5)))
    if "equitoken" in dataset_name:
        lognorm_order = torch.repeat_interleave(lognorm_order,len(curriculum)//len(lognorm_order), dim=0)
        
    plotting.plot_per_document_in_order(df, lognorm_order)
    upload(lognorm_order, "lognorm")


    lognorm_order_top_50_pct = get_order_top_n_pct(50,reweight_df(df, influence_cols, lognorm_filter(len(influence_cols), s=1, loc=0, scale=0.5)))
    if "equitoken" in dataset_name:
        lognorm_order_top_50_pct = torch.repeat_interleave(lognorm_order_top_50_pct,len(curriculum)//len(lognorm_order_top_50_pct), dim=0)
    plotting.plot_per_document_in_order(df, lognorm_order_top_50_pct)

    upload(lognorm_order_top_50_pct, "lognorm_top_50_pct")



    torch.manual_seed(42)
    lognorm_order_top_50_pct_shuffled= torch.stack([row[torch.randperm(row.shape[0])] for row in lognorm_order_top_50_pct])
    if "equitoken" in dataset_name:
        lognorm_order_top_50_pct_shuffled = torch.repeat_interleave(lognorm_order_top_50_pct_shuffled,len(curriculum)//len(lognorm_order_top_50_pct_shuffled), dim=0)
    plotting.plot_per_document_in_order(df, lognorm_order_top_50_pct_shuffled)
    upload(lognorm_order_top_50_pct_shuffled, "lognorm_top_50_pct_shuffled")


    dirac_order = get_order_full(reweight_df(df, influence_cols, dirac_filter(len(influence_cols))))
    if "equitoken" in dataset_name:
        dirac_order = torch.repeat_interleave(dirac_order,len(curriculum)//len(dirac_order), dim=0)
    plotting.plot_per_document_in_order(df, dirac_order)
    upload(dirac_order, "dirac")



    get_order_positive_only = lambda df: [torch.tensor(df[checkpoint][df[checkpoint] >= 0].sort_values(ascending=False).index.to_numpy()) for checkpoint in influence_cols]
    positive_only_order = get_order_positive_only(reweight_df(df, influence_cols, dirac_filter(len(influence_cols))))
    if "equitoken" in dataset_name:
        positive_only_order = [positive_only_order[i] for i in torch.arange(0,len(positive_only_order)).repeat_interleave(len(curriculum)//len(positive_only_order))]
    plotting.plot_per_document_in_order(df, positive_only_order)
    upload(positive_only_order, "dirac_positive_only")



    epochs = torch.tensor_split(torch.tensor(df.sort_values(by="total", ascending=False)["total"].index.to_numpy()), len(influence_cols))
    epochs = [epoch.repeat(10) for epoch in epochs]
    torch.manual_seed(42)
    influential_examples_early_order = [epoch[torch.randperm(epoch.shape[0])] for epoch in epochs]
    plotting.plot_per_document_in_order(df, influential_examples_early_order)
    upload(influential_examples_early_order, "influential_examples_early")



    halfs = torch.tensor_split(torch.tensor(df.sort_values(by="total", ascending=False)["total"].index.to_numpy()), 2)
    halfs = [half.repeat(10) for half in halfs]
    torch.manual_seed(42)
    influential_examples_first_half_order = list(torch.tensor_split(torch.cat([half[torch.randperm(len(half))] for half in halfs]), len(influence_cols)))
    plotting.plot_per_document_in_order(df, influential_examples_first_half_order)
    upload(influential_examples_first_half_order, "influential_examples_first_half")


    halfs = torch.tensor_split(torch.tensor(df.sort_values(by="total", ascending=False)["total"].index.to_numpy()), 2)
    halfs = [half.repeat(10) for half in halfs]
    torch.manual_seed(42)
    influential_examples_first_half_order = list(torch.tensor_split(torch.cat([half[torch.randperm(len(half))] for half in halfs]), len(influence_cols)))
    plotting.plot_per_document_in_order(df, influential_examples_first_half_order)
    upload(influential_examples_first_half_order, "influential_examples_first_half")


    thirds = torch.tensor_split(torch.tensor(df.sort_values(by="total", ascending=False)["total"].index.to_numpy()), 3)
    thirds = [third.repeat(10) for third in thirds]
    torch.manual_seed(42)
    influential_examples_first_third_order = list(torch.tensor_split(torch.cat([third[torch.randperm(len(third))] for third in thirds]), len(influence_cols)))
    plotting.plot_per_document_in_order(df, influential_examples_first_third_order)
    upload(influential_examples_first_third_order, "influential_examples_first_third")


    torch.manual_seed(42)
    chunks = torch.tensor_split(torch.tensor(df.sort_values(by="total", ascending=False)["total"].index.to_numpy()), len(influence_cols))
    beneficial_chunks = chunks[0:len(chunks)//2]
    harmful_chunks = chunks[len(chunks)//2:]
    beneficial_chunks_start = [beneficial_chunk.repeat(10) for beneficial_chunk in beneficial_chunks]
    harmful_chunks = [harmful_chunk.repeat(5) for harmful_chunk in harmful_chunks]
    beneficial_chunks_end = [beneficial_chunk.repeat(5) for beneficial_chunk in beneficial_chunks]
    influential_examples_sandwich = list(torch.tensor_split(torch.cat([chunk[torch.randperm(len(chunk))] for chunk in beneficial_chunks_start + harmful_chunks + beneficial_chunks_end]), len(influence_cols)))
    plotting.plot_per_document_in_order(df, influential_examples_sandwich)
    upload(influential_examples_sandwich, "influential_examples_sandwich")


FileNotFoundError: [Errno 2] No such file or directory: './mean_influence/stratified_10m_curriculum_random/stratified_10m_curriculum_train_stratified_10m_curriculum_train'